# Physics Informed Neural Networks

This notebook implements PINNs from Raissi et al. 2017. Specifically, the notebook adapts the code implementation of Data-Driven Solutions of Nonlinear Partial Differential Equations from https://github.com/maziarraissi/PINNs, and the code by Michael Ito, to solve the force-field equation for solar modulation of cosmic rays. Michael Ito's code adaption use the TF2 API where the main mechanisms of the PINN arise in the train_step function efficiently computing higher order derivatives of custom loss functions through the use of the GradientTape data structure. 

In this application, our PINN is $h(r, p) = \frac{\partial f}{\partial r} + \frac{RV}{3k} \frac{\partial f}{\partial p}$ where $k=\beta(p)k_1(r)k_2(r)$ and $\beta = \frac{p}{\sqrt{p^2 + M^2}}$. We will approximate $f(r, p)$ using a neural network.

We have no initial data, but our boundary data will be given by $f(r_{HP}, p) = \frac{J(r_{HP}, T)}{p^2} = \frac{(T+M)^\gamma}{p^2}$, where $r_{HP} = 120$ AU (i.e. the radius of Heliopause), $M=0.938$ GeV, $\gamma$ is between $-2$ and $-3$, and $T = \sqrt{p^2 + M^2} - M$. Or, vice versa, $p = \sqrt{T^2 + 2TM}$.


In [7]:
# Imports
import numpy as np
import scipy.io
import tensorflow as tf

from pyDOE import lhs

import matplotlib
import matplotlib.pyplot as plt

tf.config.list_physical_devices(device_type=None)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

## Load the Data

The data was gathered from https://github.com/maziarraissi/PINNs/tree/master/main/Data. 

In [8]:
# Constants
noise = 0.0   
M = 0.938 # GeV
gamma = -2.5 # Between -2 and -3

# Domain bounds
lb = np.array([0.0433, 1]) # (p, r) in (GeV, AU)
ub = np.array([1000.9376, 120]) # (p, r) in (GeV, AU)

# Assign number of data points
N_b = 50 # number of boundary data points
N_h = 20000 # number of collocation data points

# Create intial r, p, and T data
r = np.linspace(1, 120, 201) # r values
p = np.linspace(0.0433, 1000.9376, 256) # p values
T = np.linspace(0.001, 1000, 256) # T values
# print(r, p, T)
print(r.shape, p.shape, T.shape)

# Create boundary f data (f at r_HP)
f_exact = ((T + M)**gamma)/(p**2)
print(f_exact.shape)

P, R = np.meshgrid(p, r)
# print(len(P), len(R))
# print(P, R)

# Flatten and transpose data for ML
P_star = np.hstack((P.flatten()[:,None], R.flatten()[:,None]))

(201,) (256,) (256,)
(256,)


## PINN Class

The PINN class subclasses the Keras Model so that we can implement our custom fit and train_step functions.

Michael advises working through the TF2 API introduction to Gradients and Autodifferentiation in tensorflow https://www.tensorflow.org/guide/autodiff and Advanced Autodifferentiation in tensorflow https://www.tensorflow.org/guide/advanced_autodiff. These are the main data structures for the PINN used in the train_step function.

In [13]:
'''
Description: Defines the class for a PINN model implementing train_step, fit, and predict functions. Note, it is necessary 
to design each PINN seperately for each system of PDEs since the train_step is customized for a specific system. 
This PINN in particular solves the force-field equation for solar modulation of cosmic rays. Once trained, the PINN can predict the solution space given 
domain bounds and the input space. 
'''
class PINN(tf.keras.Model):
    def __init__(self, inputs, outputs, lower_bound, upper_bound, p, r, f_boundary, n_samples=20000, n_boundary=50):
        super(PINN, self).__init__(inputs=inputs, outputs=outputs)
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.p = p
        self.r = r
        self.f_boundary = f_boundary
        self.n_samples = n_samples
        self.n_boundary = n_boundary
        
    '''
    Description: A system of PDEs are determined by 2 types of equations: the main partial differential equations 
    and the boundary value equations. These two equations will serve as loss functions which 
    we train the PINN to satisfy. If a PINN can satisfy BOTH equations, the system is solved. Since there are 2 types of 
    equations (PDE, Boundary Value), we will need 2 types of inputs. Each input is composed of a spatial 
    variable 'r' and a temporal variable 'p'. The different types of (p, r) pairs are described below.
    
    Inputs: 
        p, r: (batchsize, 1) shaped arrays : These inputs are used to derive the main partial differential equation loss.
        Train step first feeds (p, r) through the PINN for the forward propagation. This expression is PINN(p, r) = f. 
        Next, the partials f_p and f_r are obtained.We utilize TF2s GradientTape data structure to obtain all partials. 
        Once we obtain these partials, we can compute the main PDE loss and optimize weights wrt. to the loss. 
        
        p_boundary, r_boundary : (boundary_batchsize, 1) shaped arrays : These inputs are used to derive the boundary value
        equations. The boundary value loss relies on target data (**not an equation**), so we can just measure the MSE of 
        PINN(p_boundary, r_boundary) = f_pred_boundary and boundary_f.
        
        f_boundary: (boundary_batchsize, 1) shaped arrays : This is the target data for the boundary value inputs
        
    Outputs: total_loss, pinn_loss, boundary_loss
    '''
    def train_step(self, p, r, p_boundary, r_boundary, f_boundary):
        with tf.GradientTape(persistent=True) as t2: 
            with tf.GradientTape(persistent=True) as t1: 
                # Forward pass P (PINN data)
                P = tf.concat((p, r), axis=1)
                f = self.tf_call(P)

                # Forward pass P_boundary (boundary condition data)
                P_boundary = tf.concat((p_boundary, r_boundary), axis=1)
                f_pred_boundary = self.tf_call(P_boundary)

                # Calculate boundary loss
                boundary_loss = tf.math.reduce_mean(tf.math.square(f_pred_boundary - f_boundary))

            # Calculate first-order PINN gradients
            f_p = t1.gradient(f, p)
            f_r = t1.gradient(f, r)

        # Calculate resulting loss = PINN loss + boundary loss
        pinn_loss = self.pinn_loss(f, r, p, f_p, f_r)
        total_loss = pinn_loss + boundary_loss
        
        # Backpropagation
        gradients = t2.gradient(total_loss, self.trainable_variables)
        print(gradients)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        
        # Return losses
        return total_loss.numpy(), pinn_loss.numpy(), boundary_loss.numpy()
    
    '''
    Description: The fit function used to iterate through epoch * steps_per_epoch steps of train_step. 
    
    Inputs: 
        P_predict: (N, 2) array: Input data for entire spatial and temporal domain. Used for vizualization for
        predictions at the end of each epoch. Michael created a very pretty video file with it. 
        
        batchsize: batchsize for (p, r) in train step
        
        initial_batchsize: batchsize for (x_initial, t_initial) in train step
        
        boundary_batchsize: batchsize for (x_lower, t_boundary) and (x_upper, t_boundary) in train step
        
        epochs: epochs
    
    Outputs: Losses for each equation (PDE, Initial Value, Boundary Value), and predictions for each epoch.
    '''
    def fit(self, P_predict, batchsize=64, boundary_batchsize=16, epochs=20):
        # Initialize losses as zeros
        steps_per_epoch = np.ceil(self.n_samples / batchsize).astype(int)
        total_pinn_loss = np.zeros((epochs, ))
        total_boundary_loss = np.zeros((epochs, ))
        total_predictions = np.zeros((51456, 1, epochs)) # why 51456??
        
        # For each epoch, sample new values in the PINN and boundary areas and pass them to train_step
        for epoch in range(epochs):
            # Reset loss variables
            pinn_loss = np.zeros((steps_per_epoch,))
            boundary_loss = np.zeros((steps_per_epoch,))
            
            # For each step, get PINN and boundary variables and pass them to train_step
            for step in range(steps_per_epoch):
                # Get PINN p and r variables via uniform distribution sampling between lower and upper bounds
                p = tf.Variable(tf.random.uniform((batchsize, 1), minval=self.lower_bound[0], maxval=self.upper_bound[0]))
                r = tf.Variable(tf.random.uniform((batchsize, 1), minval=self.lower_bound[1], maxval=self.upper_bound[1]))
                
                # Get p_boundary and r_boundary variables by randomly sampling along r = r_HP = 120 AU
                p_idx = np.expand_dims(np.random.choice(self.f_boundary.shape[0], boundary_batchsize, replace=False), axis=1)
                p_boundary = self.p[p_idx]
                r_boundary = np.zeros((boundary_batchsize, 1)) + 120
                f_boundary = self.f_boundary[p_idx]
                
                # Pass variables through the model via train_step and get losses
                total_loss = self.train_step(p, r, p_boundary, r_boundary, f_boundary)
                pinn_loss[step] = total_loss[1]
                boundary_loss[step] = total_loss[2]
            
            # Calculate and print total losses for the epoch
            total_pinn_loss[epoch] = np.sum(pinn_loss)
            total_boundary_loss[epoch] = np.sum(boundary_loss)
            print(f'Training loss for epoch {epoch}: pinn: {total_pinn_loss[epoch]:.4f}, boundary: {total_boundary_loss[epoch]:.4f}')
            
            # Get prediction variable loss by the predict function (below)
            total_predictions[:, :, epoch] = np.expand_dims(self.predict(P_predict)[1], axis=1)
        
        # Return epoch losses
        return total_pinn_loss, total_boundary_loss, total_predictions
    
    # Predict for some P's the value of the neural network f(r, p)
    def predict(self, P, batchsize=2048):
        steps_per_epoch = np.ceil(P.shape[0] / batchsize).astype(int)
        preds = np.zeros((P.shape[0], 2))
        
        # For each step calculate start and end index values for prediction data
        for step in range(steps_per_epoch):
            start_idx = step * 64
            
            # If last step of the epoch, end_idx is shape-1. Else, end_idx is start_idx + 64 
            if step == steps_per_epoch - 1:
                end_idx = P.shape[0] - 1
            else:
                end_idx = start_idx + 64
                
            # Get prediction data and calculate h
            preds[start_idx: end_idx, :] = self(P[start_idx: end_idx, :]).numpy()
            f = np.sqrt(preds[:, 0]**2 + preds[:, 1]**2)
        
        # Return prediction data and h
        return preds, f
    
    def evaluate(self, ): 
        pass
    
    # pinn_loss calculates the PINN loss by calculating the MSE of the pinn function
    @tf.function
    def pinn_loss(self, f, r, p, f_p, f_r): 
        R = 1 # unknown
        V = 400 # km/s
        M = 0.938 # GeV
        k_0 = 1 # km^2/s
        k = (p/tf.math.sqrt(tf.math.square(p) + tf.math.square(M)))*k_0*r*p
        l_f = tf.math.reduce_mean(tf.math.square(f_r + ((R*V)/(3*k))*f_p))
        
        return l_f
    
    # tf_call passes inputs through the neural network
    @tf.function
    def tf_call(self, inputs): 
        return self.call(inputs, training=True)

In [14]:
# Define neural network as 6 layers (4 hidden), with activation functions of tanh for hidden layers and linear for the output layer
inputs = tf.keras.Input((2))
x_ = tf.keras.layers.Dense(100, activation='tanh')(inputs)
x_ = tf.keras.layers.Dense(100, activation='tanh')(x_)
x_ = tf.keras.layers.Dense(100, activation='tanh')(x_)
x_ = tf.keras.layers.Dense(100, activation='tanh')(x_)
outputs = tf.keras.layers.Dense(2, activation='linear')(x_)

# Get keras Adam optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-4)

# Define the PINN using the model defined above
pinn = PINN(inputs=inputs, outputs=outputs, lower_bound=lb, upper_bound=ub, 
            p=p, r=r, f_boundary=f_exact)

# Compile the PINN and get loss and prediction outputs
pinn.compile(optimizer="adam")
pinn_loss, boundary_loss, predictions = pinn.fit(P_predict=P_star, batchsize=2048, 
                                               boundary_batchsize=64, epochs=1200)

[None, None, None, None, None, None, None, None, None, None]


ValueError: No gradients provided for any variable: (['dense_15/kernel:0', 'dense_15/bias:0', 'dense_16/kernel:0', 'dense_16/bias:0', 'dense_17/kernel:0', 'dense_17/bias:0', 'dense_18/kernel:0', 'dense_18/bias:0', 'dense_19/kernel:0', 'dense_19/bias:0'],). Provided `grads_and_vars` is ((None, <tf.Variable 'dense_15/kernel:0' shape=(2, 100) dtype=float32, numpy=
array([[-0.18240786, -0.12136189,  0.10202342, -0.11690237,  0.14446056,
         0.12863079,  0.22357962, -0.21971944, -0.21990113, -0.12652513,
         0.20022923,  0.12606889,  0.2325086 ,  0.22600067, -0.11639403,
         0.20897132, -0.17634717,  0.13730332, -0.06520849,  0.13826415,
        -0.1309739 ,  0.06028718, -0.19411199, -0.12039003, -0.00083759,
        -0.08642328,  0.20541418, -0.14166847,  0.01872501,  0.20606712,
        -0.06493215,  0.09048954, -0.22403544,  0.03089866,  0.01542372,
        -0.22834022,  0.10889909, -0.07259865, -0.06819643, -0.09579815,
        -0.14678124,  0.23208746, -0.14895663, -0.08

In [ ]:
import pickle as pkl

# Save loss data, prediction data, and h-function approximation
with open('./figures/pinn_loss.pkl', 'wb') as file:
    pkl.dump(pinn_loss, file)
    
with open('./figures/boundary_loss.pkl', 'wb') as file:
    pkl.dump(boundary_loss, file)
    
with open('./figures/predictions.pkl', 'wb') as file:
    pkl.dump(predictions, file)
    
with open('./figures/true.pkl', 'wb') as file:
    pkl.dump(h_star, file)